In [3]:
import gymnasium as gym
import gym_trading_env
import wandb
import pandas as pd
import numpy as np
from wandb.integration.sb3 import WandbCallback
from sb3_contrib import RecurrentPPO
from utils import reward_function_updated

# --- 2. TRAITEMENT (Inchangé) ---
def calculate_indicators(df):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['feature_rsi'] = 100 - (100 / (1 + rs))
    df['feature_rsi'] = df['feature_rsi'] / 100.0

    exp1 = df['close'].ewm(span=12, adjust=False).mean()
    exp2 = df['close'].ewm(span=26, adjust=False).mean()
    slow_macd = exp1 - exp2
    fast_macd = slow_macd.ewm(span=9, adjust=False).mean()
    df['feature_macd_histogram'] = slow_macd - fast_macd
    df['feature_macd_fast'] = fast_macd
    df['feature_macd_slow'] = slow_macd

    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['feature_atr'] = true_range.rolling(14).mean() / df['close']

    df['feature_return'] = df['close'].pct_change()


    return df.dropna()

def preprocess(df):
    df = df.sort_index().dropna().drop_duplicates()
    return calculate_indicators(df)

def run_env(position_range, learning_rate, ent_coef, gamma, gae_lambda, batch_size):
    config = {
        "policy_type": "MlpLstmPolicy",
        "total_timesteps": 300_000,    # On allonge encore, l'Alpha est dur à trouver
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "batch_size": batch_size, # Batch plus gros pour lisser le bruit des returns
        "n_steps": 2048,
        "position_range": position_range,
        "gae_lambda": gae_lambda,
        "gamma": gamma,
        "project_name": "RL-Trading-Project",
        "run_name": f"{learning_rate}_{ent_coef}_{position_range}_{gamma}_{gae_lambda}_{batch_size}",
        }

    run = wandb.init(
        project=config["project_name"],
        name=config["run_name"],
        config=config,
        sync_tensorboard=True,
        monitor_gym=True,
        save_code=True,
    )

    env = gym.make(
        "MultiDatasetTradingEnv",
        dataset_dir="./data/*.pkl",
        preprocess=preprocess,
        portfolio_initial_value=1000,
        trading_fees=0.1/100,
        borrow_interest_rate=0.02/100/24,
        reward_function=reward_function_updated,
        position_range=config["position_range"]
    )

    # Création de la callback Wandb
    wandb_callback = WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"models/{run.id}",
        verbose=2,
    )

    model = RecurrentPPO(
        config["policy_type"],
        env,
        verbose=1,
        learning_rate=config["learning_rate"],
        ent_coef=config["ent_coef"],
        batch_size=config["batch_size"],
        n_steps=config["n_steps"],
        tensorboard_log=f"runs/{run.id}",
        gamma=config["gamma"],
        gae_lambda=config["gae_lambda"]
    )

    print(f"--- Démarrage Alpha Hunter ---")
    print(f"Objectif : Battre le Buy & Hold (Reward = Return - Market)")
    print(f"Exploration forcée (Ent_coef={config['ent_coef']})")

    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=wandb_callback
    )

    print("Fin de l'apprentissage")

    model.save("recurrent_ppo_alpha_hunter")

    obs, info = env.reset()
    done, truncated = False, False

    while not (done or truncated):
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)

    metrics = env.unwrapped.get_metrics()
    print("Métriques finales :", metrics)

    market_return = float(metrics.get("Market Return", "0%").replace('%',''))
    portfolio_return = float(metrics.get("Portfolio Return", "0%").replace('%', ''))
    metrics_float = {'Market Return (%): ': market_return, 'Portfolio Return (%): ': portfolio_return}
    wandb.log(metrics_float)

    wandb.finish()

In [ ]:
# Liste des différents hyperparamètres à tester
learning_rates = [1e-4, 1e-3] # Taux d'apprentissage (1e-4 et 1e-3)
ent_coefs = [0.01, 0.05] # Coefficients d'entropie pour encourager l'exploration (0.01 et 0.05)
position_ranges = [(-1,1), (-0.5, 0.5), (0,0.8)] # Différentes plages de positions [-1,1], [-0.5,0.5], [0,0.8]
gammas = [0.9, 0.95, 0.99] # Facteurs d'actualisation (0.9, 0.95, 0.99)
gae_lambdas = [0.01, 0.05, 0.1] # Paramètres GAE (0.01, 0.05, 0.1)
batch_sizes = [128, 256] # Tailles de batch (128 et 256)

# Boucle sur toutes les combinaisons d'hyperparamètres
for lr in learning_rates:
    for ent in ent_coefs:
        for pos_range in position_ranges:
            for gamma in gammas:
                for gae_lambda in gae_lambdas:
                    for batch_size in batch_sizes:
                        config = {
                            "policy_type": "MlpLstmPolicy",
                            "total_timesteps": 200_000,    # On allonge encore, l'Alpha est dur à trouver
                            "learning_rate": lr,
                            "ent_coef": ent,
                            "batch_size": batch_size, # Batch plus gros pour lisser le bruit des returns
                            "n_steps": 2048,
                            "position_range": pos_range,
                            "gae_lambda": gae_lambda,
                            "gamma": gamma,
                            "project_name": "RL-Trading-Project",
                            "run_name": f"{lr}_{ent}_{pos_range}_{gamma}_{gae_lambda}_{batch_size}",
                        }
                        run = wandb.init(
                            project=config["project_name"],
                            name=config["run_name"],
                            config=config,
                            sync_tensorboard=True,
                            monitor_gym=True,
                            save_code=True,
                        )

                        env = gym.make(
                            "MultiDatasetTradingEnv",
                            dataset_dir="./data/*.pkl",
                            preprocess=preprocess,
                            portfolio_initial_value=1000,
                            trading_fees=0.1/100,
                            borrow_interest_rate=0.02/100/24,
                            reward_function=reward_function_updated,
                            position_range=config["position_range"]
                        )

                        # Création de la callback Wandb
                        wandb_callback = WandbCallback(
                            gradient_save_freq=100,
                            model_save_path=f"models/{run.id}",
                            verbose=2,
                        )

                        model = RecurrentPPO(
                            config["policy_type"],
                            env,
                            verbose=1,
                            learning_rate=config["learning_rate"],
                            ent_coef=config["ent_coef"],
                            batch_size=config["batch_size"],
                            n_steps=config["n_steps"],
                            tensorboard_log=f"runs/{run.id}",
                            gamma=config["gamma"],
                            gae_lambda=config["gae_lambda"]
                        )

                        print(f"--- Démarrage Alpha Hunter ---")
                        print(f"Objectif : Battre le Buy & Hold (Reward = Return - Market)")
                        print(f"Exploration forcée (Ent_coef={config['ent_coef']})")

                        model.learn(
                            total_timesteps=config["total_timesteps"],
                            callback=wandb_callback
                        )

                        print("Fin de l'apprentissage")

                        model.save("recurrent_ppo_alpha_hunter")

                        obs, info = env.reset()
                        done, truncated = False, False

                        while not (done or truncated):
                            action, _states = model.predict(obs, deterministic=True)
                            obs, reward, done, truncated, info = env.step(action)

                        metrics = env.unwrapped.get_metrics()
                        print("Métriques finales :", metrics)

                        market_return = float(metrics.get("Market Return", "0%").replace('%',''))
                        portfolio_return = float(metrics.get("Portfolio Return", "0%").replace('%', ''))
                        metrics_float = {'Market Return (%): ': market_return, 'Portfolio Return (%): ': portfolio_return}
                        wandb.log(metrics_float)

                        wandb.finish()




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
--- Démarrage Alpha Hunter ---
Objectif : Battre le Buy & Hold (Reward = Return - Market)
Exploration forcée (Ent_coef=0.01)


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Logging to runs/i9bwm7ab\RecurrentPPO_1
-----------------------------
| time/              |      |
|    fps             | 1032 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
Market Return : 39.25%   |   Portfolio Return : -94.83%   |   
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.48e+03     |
|    ep_rew_mean          | -245         |
| time/                   |              |
|    fps                  | 599          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0012395335 |
|    clip_fraction        | 0.000635     |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | 0.023        |
|    learning_rate        | 0.0001       |